In [2]:
# Import Dependencies
import numpy as np
import os
import pandas as pd


In [3]:
# Path to file directory and variables for the two files.
file_dir = os.path.join("..", "Data")

# The Kaggle metadata
kaggle_file = f'{file_dir}/movies_metadata.csv'

# The MovieLens rating data
ratings_file = f'{file_dir}/ratings_small.csv'

In [4]:
# Import Kaggle metadata, and MovieLens rating data (from Kaggle)

# Read in the kaggle metadata and MovieLens ratings CSV files as Pandas DataFrames.
kaggle_metadata = pd.read_csv(kaggle_file, low_memory=False)
ratings = pd.read_csv(ratings_file)

# Import Viewer Movies and Ratings (from IMDB)
#viewer_movies = 

In [5]:
# Check DataFrames
kaggle_metadata.head()

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0


In [ ]:
# Clean the Kaggle metadata.
# Keep only Non-Adult Movies, then drop Adult Column
kaggle_metadata = kaggle_metadata[kaggle_metadata['adult'] == 'False'].drop('adult', axis='columns')

# Convert video Column to Boolean
kaggle_metadata["video"] = kaggle_metadata["video"] == 'True'

# Convert Columns to Numeric
kaggle_metadata['budget'] = kaggle_metadata['budget'].astype(int)
kaggle_metadata['id'] = pd.to_numeric(kaggle_metadata['id'], errors='raise')
kaggle_metadata['popularity'] = pd.to_numeric(kaggle_metadata['popularity'], errors='raise')

# Convert release_date to datetime
kaggle_metadata['release_date'] = pd.to_datetime(kaggle_metadata['release_date'])


# Filter the movies DataFrame for specific columns.
movies_df = movies_df.loc[:, ['imdb_id','id','title_kaggle','original_title','tagline','belongs_to_collection','url','imdb_link',
   'runtime','budget_kaggle','revenue','release_date_kaggle','popularity','vote_average','vote_count',
   'genres','original_language','overview','spoken_languages','Country',
   'production_companies','production_countries','Distributor',
   'Producer(s)','Director','Starring','Cinematography','Editor(s)','Writer(s)','Composer(s)','Based on'
  ]]

# Rename the columns in the movies DataFrame.
movies_df.rename({'id':'kaggle_id',
      'title_kaggle':'title',
      'url':'wikipedia_url',
      'budget_kaggle':'budget',
      'release_date_kaggle':'release_date',
      'Country':'country',
      'Distributor':'distributor',
      'Producer(s)':'producers',
      'Director':'director',
      'Starring':'starring',
      'Cinematography':'cinematography',
      'Editor(s)':'editors',
      'Writer(s)':'writers',
      'Composer(s)':'composers',
      'Based on':'based_on'
     }, axis='columns', inplace=True)

# Transform and merge the ratings DataFrame.
# Count Ratings
rating_counts = ratings.groupby(['movieId','rating'], as_index=False).count() \
    .rename({'userId':'count'}, axis=1)

# Pivot Ratings Count Data
rating_counts = ratings.groupby(['movieId','rating'], as_index=False).count() \
    .rename({'userId':'count'}, axis=1) \
    .pivot(index='movieId',columns='rating', values='count')

# Rename Ratings Count Columns
rating_counts.columns = ['rating_' + str(col) for col in rating_counts.columns]

# Merge Rating Counts into movies_df
movies_with_ratings_df = pd.merge(movies_df, rating_counts, left_on='kaggle_id', right_index=True, how='left')

# Fill in Missing Ratings with Zeroes
movies_with_ratings_df[rating_counts.columns] = movies_with_ratings_df[rating_counts.columns].fillna(0)

